# 可转债套利回测分析

本 Notebook 使用本项目中的 **真实** 回测器 `cb_arb.backtest.CBArbBacktester`，复现与 `examples.run_simple_backtest` 相同的数据与配置，展示完整回测流程与结果。

所有输入与 `run_simple_backtest` 一致，输出为 `backtester.run()` 的真实返回值。

## 1. 路径与导入

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

cwd = os.getcwd()
project_root = cwd if os.path.isdir(os.path.join(cwd, 'src')) else os.path.abspath(os.path.join(cwd, '..'))
src_dir = os.path.join(project_root, 'src')
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

from cb_arb.params import ConvertibleBondContract, TermStructure, CreditCurve
from cb_arb.signals import MispricingSignalConfig
from cb_arb.backtest import CBArbBacktester

# 导入 Notebook 工具函数（用于保存输出文件）
notebooks_dir = os.path.join(project_root, 'notebooks')
if notebooks_dir not in sys.path:
    sys.path.insert(0, notebooks_dir)
from notebook_utils import save_figure, get_figures_dir, get_data_dir

print("项目根目录:", project_root)
print("src 目录已添加到 Python 路径:", src_dir)
print("输出目录:", os.path.join(project_root, 'output'))
print("CBArbBacktester 导入成功。")

## 2. 生成与 run_simple_backtest 完全一致的数据

GBM 股价与带噪声的可转债市场价，与示例脚本中逻辑相同。

In [ ]:
def simulate_gbm_path(S0, r, q, vol, dates, seed=42):
    dt = 1.0 / 252.0
    n = len(dates)
    rng = np.random.default_rng(seed)
    shocks = rng.normal(0.0, np.sqrt(dt), size=n)
    prices = [S0]
    for eps in shocks[1:]:
        s_prev = prices[-1]
        s_new = s_prev * np.exp((r - q - 0.5 * vol**2) * dt + vol * eps)
        prices.append(s_new)
    return pd.Series(prices, index=dates, name='stock')

dates = pd.date_range("2020-01-01", periods=250, freq="B")
S0, r, q, vol = 100.0, 0.02, 0.01, 0.25
face, conversion_ratio = 100.0, 1.0

stock_series = simulate_gbm_path(S0, r, q, vol, dates)
years = np.linspace(0.0, 1.0, len(dates))
bond_floor = face * np.exp(-0.01 * years)
conv_part = 0.4 * conversion_ratio * stock_series.values
cb_theoretical = bond_floor + conv_part
rng = np.random.default_rng(123)
noise = 0.02 * rng.standard_normal(len(dates))
cb_market = pd.Series(cb_theoretical * (1.0 + noise), index=dates, name='cb_market')

print("股票路径与可转债市场价已生成，长度:", len(stock_series))

## 3. 构建回测器并运行（与 run_simple_backtest 相同配置）

In [ ]:
contract = ConvertibleBondContract(
    face_value=face,
    coupon_rate=0.03,
    maturity=3.0,
    conversion_ratio=conversion_ratio,
    issue_price=100.0,
    call_price=None,
    put_price=None,
    coupon_freq=2,
)
r_curve = TermStructure(rate_fn=lambda t: r)
q_curve = TermStructure(rate_fn=lambda t: q)
credit_curve = CreditCurve(spread_fn=lambda t: 0.03)
steps = 50
signal_cfg = MispricingSignalConfig(lookback=40, entry_z=-1.5, exit_z=-0.5)
initial_cb_face = 100_000.0

backtester = CBArbBacktester(
    contract=contract,
    r_curve=r_curve,
    q_curve=q_curve,
    credit_curve=credit_curve,
    vol=vol,
    steps=steps,
    signal_cfg=signal_cfg,
    initial_cb_face=initial_cb_face,
)

result = backtester.run(cb_market_price=cb_market, stock_price=stock_series)

print("回测结果 DataFrame 形状:", result.shape)
print("列:", list(result.columns))

## 4. 查看关键列与尾部统计（真实 run() 输出）

In [ ]:
cols = ['mispricing', 'zscore', 'signal', 'pnl', 'cum_pnl']
print("最后 10 行:")
print(result[cols].tail(10).to_string())
print("\n累计 PnL 统计: min=%.2f, max=%.2f, 最终=%.2f" % (
    result['cum_pnl'].min(), result['cum_pnl'].max(), result['cum_pnl'].iloc[-1]))

## 5. 累计收益与股价对比图（与 run_simple_backtest 一致）

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 5))
ax1.plot(result.index, result['cum_pnl'], label='CB Arb Cum PnL', color='C0')
ax1.set_ylabel('Cum PnL', color='C0')
ax1.tick_params(axis='y', labelcolor='C0')

ax2 = ax1.twinx()
ax2.plot(stock_series.index, stock_series.values, label='Stock Price', color='C1', alpha=0.5)
ax2.set_ylabel('Stock Price', color='C1')
ax2.tick_params(axis='y', labelcolor='C1')

fig.suptitle('Convertible Arbitrage Backtest (CBArbBacktester.run 真实输出)')
fig.tight_layout()

# 保存图片
save_figure(fig, '05_backtest_analysis_result')

# 保存回测结果数据
from datetime import datetime
data_dir = get_data_dir()
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_path = data_dir / f'backtest_result_{timestamp}.csv'
result.to_csv(csv_path, encoding='utf-8-sig')
print(f'回测结果数据已保存到: {csv_path}')

plt.show()